In [ ]:
pip install arviz

     |████████████████████████████████| 1.6MB 6.8MB/s 
     |████████████████████████████████| 768kB 29.0MB/s 
     |████████████████████████████████| 4.7MB 37.8MB/s 
     |████████████████████████████████| 317kB 46.2MB/s 
  Found existing installation: xarray 0.15.1
    Uninstalling xarray-0.15.1:
      Successfully uninstalled xarray-0.15.1


In [ ]:
import pystan
import numpy as np
import matplotlib.pyplot as plt
import arviz
import pickle
import cmdstanpy
import json
import pandas as pd

In [ ]:
with open('/content/drive/MyDrive/TFM-Crowd learning/multinomial.json') as json_file:
    data = json.load(json_file)

In [ ]:
dades={'Tasca': data['t_A'], 'Worker': data['w_A'], 'Anotació': data['ann'] }
df = pd.DataFrame(data=dades)
df

,Tasca,Worker,Anotació
0,0,14,2
1,0,13,2
2,0,4,2
3,0,2,2
4,0,7,1
...,...,...,...
9995,999,17,2
9996,999,2,2
9997,999,13,2
9998,999,14,2


In [ ]:
sm_ext = pystan.StanModel(file='/content/drive/MyDrive/TFM-Crowd learning/Fitxers STAN/extern_classes.stan')


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ccda708da43115a83f1bd313f51b2f37 NOW.


In [ ]:
w = len(np.unique(df['Worker']))
t = len(np.unique(df['Tasca']))
a = len(df['Anotació'])
k = len(np.unique(df['Anotació']))
num_classes_externes = 1
k2=k+num_classes_externes
t_A = np.array(df['Tasca'])+1
w_A = np.array(df['Worker'])+1
ann = np.array(df['Anotació'])+1
tau_prior = np.array([1,1,1])
pi_prior = np.ones((k,k2))

dades_sm_ext = {'w':w,'t':t,'a':a,'k':k,'k2':k2,'t_A':t_A,'w_A':w_A,'ann':ann,'tau_prior':tau_prior,'pi_prior':pi_prior} 

In [ ]:
inicial = np.array([[0.91,0.03,0.03,0.03],
                    [0.03,0.91,0.03,0.03],
                    [0.03,0.03,0.91,0.03]])

In [ ]:
pi_ini = np.zeros((w,k,k2))
for yw in range(w):
  pi_ini[yw] = inicial
dades_inicials_gen = {'pi':pi_ini}

In [ ]:
optim = sm_ext.optimizing(data=dades_sm_ext,init=dades_inicials_gen)
optim

OrderedDict([('tau', array([0.1125492 , 0.29707459, 0.59037621])),
             ('pi',
              array([[[8.50400100e-01, 7.91348876e-02, 7.04650123e-02, 3.44266282e-12],
                      [2.19057404e-01, 5.88754361e-01, 1.92188225e-01, 9.84673559e-09],
                      [1.47670077e-01, 9.12949196e-02, 7.61034953e-01, 5.09124678e-08]],
              
                     [[6.54304459e-01, 1.26588446e-01, 2.19106960e-01, 1.34797369e-07],
                      [3.07676536e-01, 5.17248083e-01, 1.75075365e-01, 1.63881191e-08],
                      [1.30591696e-01, 1.10734431e-01, 7.58673823e-01, 4.97214194e-08]],
              
                     [[6.24033193e-01, 8.48437572e-02, 2.91122921e-01, 1.29077593e-07],
                      [2.13772278e-01, 6.02179037e-01, 1.84048669e-01, 1.59265858e-08],
                      [7.40930215e-02, 1.08487321e-01, 8.17419608e-01, 4.93351936e-08]],
              
                     [[6.92217930e-01, 9.32569728e-02, 2.14524988e-01, 1.